In [29]:
# import neccessary dependencies
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import accuracy_score

# Import the data

In [30]:
# training data
#train = pd.read_csv(r"C:\Users\Admin\OneDrive\Documents\Webdev\hatespeech\train.csv", nrows=5000)

# testing data
#test = pd.read_csv(r"C:\Users\Admin\OneDrive\Documents\Webdev\hatespeech\test.csv", nrows=5000)


# #training data
train = pd.read_csv(r"C:\Users\jmest\Documents\Files\C4S2\Thesis\ThesisFiles-main\ThesisFiles-main\hatespeech\train.csv", nrows=100)

# #testing data
test = pd.read_csv(r"C:\Users\jmest\Documents\Files\C4S2\Thesis\ThesisFiles-main\ThesisFiles-main\hatespeech\test.csv", nrows=5000)




# Clean the data of null values

In [31]:
# drop rows with null values
train.dropna()

,text,label
0,Inaasahan na ni Vice President Jejomar Binay n...,0
1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,1
2,Salamat sa walang sawang suporta ng mga taga m...,0
3,@rapplerdotcom putangina mo binay TAKBO PA,1
4,"Binay with selective amnesia, forgetting about...",0
...,...,...
95,Tangina ka Enrile. Di naman masyadong halatang...,1
96,@suigeneris_05 Errata: Binay pa rin ang may pi...,1
97,Introduction mo Mar Roxas panay paninira lang ...,1
98,I VOTE ROXAS #Halalan2016 Mar Daang Matuwid RO...,0


In [32]:
#check for null values in train
train.isnull().sum()

text     0
label    0
dtype: int64

In [33]:
# check for 0 values in train
sum(train["label"] == 0)

63

In [34]:
# check for 1 values in train
sum(train["label"] == 1)

37

# Clean the data of unwated Text and Characters

In [35]:
# remove special characters using the regular expression library

import re

#set up punctuations we want to be replaced

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [36]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
    tempArr = []
    for line in df:
        # send to tweet_processor
        tmpL = p.clean(line)
        # remove everything except letters and digits
        tmpL = re.sub(r'[^a-zA-Z0-9\s]', ' ', tmpL)
        # convert to lowercase
        tmpL = tmpL.lower()
        tempArr.append(tmpL)
    return tempArr

In [37]:
# clean training data
train_tweet = clean_tweets(train["text"])
train_tweet = pd.DataFrame(train_tweet)

In [38]:
# append cleaned tweets to the training data
train["clean_tweet"] = train_tweet

# compare the cleaned and uncleaned tweets
train.head(10)

,text,label,clean_tweet
0,Inaasahan na ni Vice President Jejomar Binay n...,0,inaasahan na ni vice president jejomar binay n...
1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,1,mar roxas tang ina tuwid na daan daw eh sya...
2,Salamat sa walang sawang suporta ng mga taga m...,0,salamat sa walang sawang suporta ng mga taga m...
3,@rapplerdotcom putangina mo binay TAKBO PA,1,putangina mo binay takbo pa
4,"Binay with selective amnesia, forgetting about...",0,binay with selective amnesia forgetting about...
5,It doesn't matter whoever won between Duterte ...,0,it doesn t matter whoever won between duterte ...
6,Nognog? Pero nognog din ang nag malasakit? Wtf...,1,nognog pero nognog din ang nag malasakit wtf...
7,#OnlyB1nay ?? #FB https://t.co/QEQnsK67Gm,1,
8,What Abi Binay said on running for Makati mayo...,0,what abi binay said on running for makati mayor
9,Srsly. How can Binay do away with no tax for t...,1,srsly how can binay do away with no tax for t...


In [39]:
sum(train['clean_tweet'] == '')

2

In [40]:
#df['Another'] = df['Another'].replace('', np.nan)
#replace all empty spaces with NaN to drop using dropna
train['clean_tweet'] = train['clean_tweet'].replace('', np.NaN)

In [41]:
train.dropna(axis='rows')

,text,label,clean_tweet
0,Inaasahan na ni Vice President Jejomar Binay n...,0,inaasahan na ni vice president jejomar binay n...
1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,1,mar roxas tang ina tuwid na daan daw eh sya...
2,Salamat sa walang sawang suporta ng mga taga m...,0,salamat sa walang sawang suporta ng mga taga m...
3,@rapplerdotcom putangina mo binay TAKBO PA,1,putangina mo binay takbo pa
4,"Binay with selective amnesia, forgetting about...",0,binay with selective amnesia forgetting about...
...,...,...,...
95,Tangina ka Enrile. Di naman masyadong halatang...,1,tangina ka enrile di naman masyadong halatang...
96,@suigeneris_05 Errata: Binay pa rin ang may pi...,1,errata binay pa rin ang may pinakamataas na d...
97,Introduction mo Mar Roxas panay paninira lang ...,1,introduction mo mar roxas panay paninira lang ...
98,I VOTE ROXAS #Halalan2016 Mar Daang Matuwid RO...,0,i vote roxas mar daang matuwid roro rising all...


In [42]:
#https://statisticsglobe.com/drop-rows-blank-values-from-pandas-dataframe-python
train['clean_tweet'] = train['clean_tweet'].replace('', float('NaN'), regex = True)

In [43]:
train.dropna(inplace= True)
train = train.reset_index(drop=True)

In [44]:
first_column = train.pop('label')
train.insert(0,'label',first_column)

In [45]:
train.head(10)

,label,text,clean_tweet
0,0,Inaasahan na ni Vice President Jejomar Binay n...,inaasahan na ni vice president jejomar binay n...
1,1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,mar roxas tang ina tuwid na daan daw eh sya...
2,0,Salamat sa walang sawang suporta ng mga taga m...,salamat sa walang sawang suporta ng mga taga m...
3,1,@rapplerdotcom putangina mo binay TAKBO PA,putangina mo binay takbo pa
4,0,"Binay with selective amnesia, forgetting about...",binay with selective amnesia forgetting about...
5,0,It doesn't matter whoever won between Duterte ...,it doesn t matter whoever won between duterte ...
6,1,Nognog? Pero nognog din ang nag malasakit? Wtf...,nognog pero nognog din ang nag malasakit wtf...
7,1,#OnlyB1nay ?? #FB https://t.co/QEQnsK67Gm,
8,0,What Abi Binay said on running for Makati mayo...,what abi binay said on running for makati mayor
9,1,Srsly. How can Binay do away with no tax for t...,srsly how can binay do away with no tax for t...


In [46]:
#total data entries for training

print(train.shape)

(98, 3)


In [47]:
#remove stopwords
import urllib.request, json 
with urllib.request.urlopen("https://raw.githubusercontent.com/stopwords-iso/stopwords-tl/master/stopwords-tl.json") as url:
    stopwords = json.loads(url.read().decode())
    print(stopwords)

['akin', 'aking', 'ako', 'alin', 'am', 'amin', 'aming', 'ang', 'ano', 'anumang', 'apat', 'at', 'atin', 'ating', 'ay', 'bababa', 'bago', 'bakit', 'bawat', 'bilang', 'dahil', 'dalawa', 'dapat', 'din', 'dito', 'doon', 'gagawin', 'gayunman', 'ginagawa', 'ginawa', 'ginawang', 'gumawa', 'gusto', 'habang', 'hanggang', 'hindi', 'huwag', 'iba', 'ibaba', 'ibabaw', 'ibig', 'ikaw', 'ilagay', 'ilalim', 'ilan', 'inyong', 'isa', 'isang', 'itaas', 'ito', 'iyo', 'iyon', 'iyong', 'ka', 'kahit', 'kailangan', 'kailanman', 'kami', 'kanila', 'kanilang', 'kanino', 'kanya', 'kanyang', 'kapag', 'kapwa', 'karamihan', 'katiyakan', 'katulad', 'kaya', 'kaysa', 'ko', 'kong', 'kulang', 'kumuha', 'kung', 'laban', 'lahat', 'lamang', 'likod', 'lima', 'maaari', 'maaaring', 'maging', 'mahusay', 'makita', 'marami', 'marapat', 'masyado', 'may', 'mayroon', 'mga', 'minsan', 'mismo', 'mula', 'muli', 'na', 'nabanggit', 'naging', 'nagkaroon', 'nais', 'nakita', 'namin', 'napaka', 'narito', 'nasaan', 'ng', 'ngayon', 'ni', 'nila',

In [48]:
#data['content2'] =data['Content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
train['rm_stpwrds'] = train['clean_tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords) ]))

In [49]:
train[['clean_tweet', 'rm_stpwrds']].head(10)

,clean_tweet,rm_stpwrds
0,inaasahan na ni vice president jejomar binay n...,inaasahan vice president jejomar binay taong
1,mar roxas tang ina tuwid na daan daw eh sya...,mar roxas tang ina tuwid daan daw eh sya nga d...
2,salamat sa walang sawang suporta ng mga taga m...,salamat sawang suporta taga makati pagbabalik ...
3,putangina mo binay takbo pa,putangina mo binay takbo
4,binay with selective amnesia forgetting about...,binay with selective amnesia forgetting about ...
5,it doesn t matter whoever won between duterte ...,it doesn t matter whoever won between duterte ...
6,nognog pero nognog din ang nag malasakit wtf...,nognog nognog nag malasakit wtf tangina mo bin...
7,,
8,what abi binay said on running for makati mayor,what abi binay said on running for makati mayor
9,srsly how can binay do away with no tax for t...,srsly how can binay do away with no tax for th...


In [50]:
#tokenization 

import nltk
from nltk.tokenize import WhitespaceTokenizer
train['tokenize'] = train['rm_stpwrds'].apply(nltk.tokenize.WhitespaceTokenizer().tokenize) 
train['tokenize'].head()



0    [inaasahan, vice, president, jejomar, binay, t...
1    [mar, roxas, tang, ina, tuwid, daan, daw, eh, ...
2    [salamat, sawang, suporta, taga, makati, pagba...
3                        [putangina, mo, binay, takbo]
4    [binay, with, selective, amnesia, forgetting, ...
Name: tokenize, dtype: object

In [51]:
train[['rm_stpwrds','tokenize']].head(10)

,rm_stpwrds,tokenize
0,inaasahan vice president jejomar binay taong,"[inaasahan, vice, president, jejomar, binay, t..."
1,mar roxas tang ina tuwid daan daw eh sya nga d...,"[mar, roxas, tang, ina, tuwid, daan, daw, eh, ..."
2,salamat sawang suporta taga makati pagbabalik ...,"[salamat, sawang, suporta, taga, makati, pagba..."
3,putangina mo binay takbo,"[putangina, mo, binay, takbo]"
4,binay with selective amnesia forgetting about ...,"[binay, with, selective, amnesia, forgetting, ..."
5,it doesn t matter whoever won between duterte ...,"[it, doesn, t, matter, whoever, won, between, ..."
6,nognog nognog nag malasakit wtf tangina mo bin...,"[nognog, nognog, nag, malasakit, wtf, tangina,..."
7,,[]
8,what abi binay said on running for makati mayor,"[what, abi, binay, said, on, running, for, mak..."
9,srsly how can binay do away with no tax for th...,"[srsly, how, can, binay, do, away, with, no, t..."


In [52]:
from nltk.stem.wordnet import WordNetLemmatizer
def lema_words(text):
  wnl=WordNetLemmatizer()
  return[wnl.lemmatize(w) for w in text]

train['lematize']=train['tokenize'].apply(lema_words)  
train[['tokenize','lematize']].sample(5)

,tokenize,lematize
31,"[haha, binay, jollibee]","[haha, binay, jollibee]"
21,"[si, senator, nancy, binay, ba, tinutukoy, mo,...","[si, senator, nancy, binay, ba, tinutukoy, mo,..."
70,"[duterte, roxas, poe, binay, santiago, rank]","[duterte, roxas, poe, binay, santiago, rank]"
3,"[putangina, mo, binay, takbo]","[putangina, mo, binay, takbo]"
78,"[say, goodbye, to, your, presidential, ambitio...","[say, goodbye, to, your, presidential, ambitio..."


# Proceeding to Training

In [53]:
# Train the SVM model
clf = svm.SVC()
clf.fit(X_train, y_train)

# Test the model on the testing set
y_pred = clf.predict(X_test)

# Evaluate the model's performance
accuracy = clf.score(X_test, y_test)
print("Accuracy:", accuracy)

NameError: name 'X_train' is not defined

In [ ]:
#training with lemmatized words
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train['lematize'], train['label'], test_size=0.3, random_state=1)


In [ ]:
# Vectorize the sentences using TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(map(' '.join, X_train))
X_test = vectorizer.transform(map(' '.join, X_test))

In [ ]:
# Train the SVM model
clf = svm.SVC()
clf.fit(X_train, y_train)

# Test the model on the testing set
y_pred = clf.predict(X_test)

# Evaluate the model's performance
accuracy = clf.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.7242105263157895


In [65]:
VOWELS = "aeiouAEIOU"
CONSONANTS = "bcdfghklmnngpqrstvwyBCDFGHKLMNNGPQRSTVWY"

""" 
	Affixes
"""
PREFIX_SET = [
	'nakikipag', 'pakikipag',
	'pinakama', 'pagpapa',
	'pinagka', 'panganga', 
	'makapag', 'nakapag', 
	'tagapag', 'makipag', 
	'nakipag', 'tigapag',
	'pakiki', 'magpa',
	'napaka', 'pinaka',
	'ipinag', 'pagka', 
	'pinag', 'mapag', 
	'mapa', 'taga', 
	'ipag', 'tiga', 
	'pala', 'pina', 
	'pang', 'naka',
	'nang', 'mang',
	'sing',
	'ipa', 'pam',
	'pan', 'pag',
	'tag', 'mai',
	'mag', 'nam',
	'nag', 'man',
	'may', 'ma',
	'na', 'ni',
	'pa', 'ka',
	'um', 'in',
	'i',
]

INFIX_SET = [
	'um', 'in',
]

SUFFIX_SET = [
	'syon','dor', 
	'ita', 'han', 
	'hin', 'ing', 
	'ang', 'ng', 
	'an', 'in', 
	'g',
]

PERIOD_FLAG = True
PASS_FLAG = False





def check_vowel(substring):
	"""
		Checks if the substring is a vowel.
			letters: substring to be tested
		returns BOOLEAN
	"""

	return all(letter in VOWELS for letter in substring)


def check_consonant(substring):
	"""
		Checks if the letter is a consonant.
			letter: substring to be tested
		returns BOOLEAN
	"""

	return all(letter in CONSONANTS for letter in substring)

def change_letter(token, index, letter):
	"""
		Replaces a letter in a token.
			token: word to be used
			index: index of the letter
			letter: letter used to replace
		returns STRING
	"""
	
	_list = list(token)
	_list[index] = letter

	return ''.join(_list)

def count_vowel(token):
	"""
		Count vowels in a given token.
			token: string to be counted for vowels
		returns INTEGER
	"""

	count = 0

	for tok in token:
		if check_vowel(tok):
			count+=1

	return count


def count_consonant(token):
	"""
		Count consonants in a given token.
			token: string to be counted for consonants
		returns INTEGER
	"""

	count = 0

	for tok in token:
		if check_consonant(tok):
			count+=1

	return count





def check_validation(token):
    with open('stemmer/validation.txt', 'r') as valid:
        data = valid.read().replace('\n', ' ').split(' ')

    return token in data



def clean_repitition(token, REPITITION):
	"""
		Checks token for repitition. (ex. nakakabaliw = nabaliw)
			token: word to be stemmed repitition
		returns STRING
	"""

	if check_validation(token):
		return token

	if len(token) >= 4:
		if check_vowel(token[0]):
			if token[0] == token[1]:
				REPITITION.append(token[0])
				return token[1:]

		elif check_consonant(token[0]) and count_vowel(token) >= 2:
			if token[0: 2] == token[2: 4] and len(token) - 2 >= 4:
				REPITITION.append(token[2:4])
				return token[2:]
			
			elif token[0: 3] == token[3: 6] and len(token) - 3 >= 4:
				REPITITION.append(token[3:6])
				return token[3:]

	return token

def clean_suffix(token, SUFFIX):
    """
    Checks token for suffixes. (ex. bigayan = bigay)
        token: word to be stemmed for suffixes
    returns STRING
    """

    SUF_CANDIDATE = []

    if check_validation(token):
        return token

    for suffix in SUFFIX_SET:
        if len(token) - len(suffix) >= 3 and count_vowel(token[0:len(token) - len(suffix)]) >= 2 and count_consonant(token[0:len(token) - len(suffix)]) >= 1:
            if token[len(token) - len(suffix): len(token)] == suffix:
                if len(suffix) == 2 and not count_consonant(token[0:len(token) - len(suffix)]) >= 1:
                    continue

                if count_vowel(token[0: len(token) - len(suffix)]) >= 2:
                    if suffix == 'ang' and check_consonant(token[-4]) \
                            and token[-4] != 'r' and token[-5] != 'u':
                        continue

                    #print(token[0: len(token) - len(suffix)] + " : " + suffix)

                    if check_validation(token[0: len(token) - len(suffix)]):
                        SUFFIX.append(suffix)
                        return token[0: len(token) - len(suffix)] + 'a' if suffix == 'ita' \
                            else token[0: len(token) - len(suffix)]

                    elif len(SUF_CANDIDATE) == 0:
                        SUF_CANDIDATE.append(suffix)
                        SUF_CANDIDATE.append(token[0: len(token) - len(suffix)])

    if (len(SUF_CANDIDATE) == 2):
        SUFFIX = SUF_CANDIDATE[0]
        return SUF_CANDIDATE[1][0: len(token) - len(SUFFIX)] + 'a' if SUFFIX == 'ita' \
            else SUF_CANDIDATE[1][0: len(token) - len(SUFFIX)]

    return token


def clean_infix(token, INFIX):
	"""
		Checks token for infixes. (ex. bumalik = balik)
			token: word to be stemmed for infixes
		returns STRING
	"""

	if check_validation(token):
		return token

	for infix in INFIX_SET:
		if len(token) - len(infix) >= 3 and count_vowel(token[len(infix):]) >= 2:
			if token[0] == token[4] and token[1: 4] == infix:
				INFIX.append(infix)
				return token[4:]

			elif token[2] == token[4] and token[1: 3] == infix:
				INFIX.append(infix)
				return token[0] + token[3:]

			elif token[1: 3] == infix and check_vowel(token[3]):
				INFIX.append(infix)
				return token[0] + token[3:]

	return token


def clean_prefix(token,	 PREFIX):
	"""
		Checks token for prefixes. (ex. naligo = ligo)
			token: word to be stemmed for prefixes
		returns STRING
	"""

	if check_validation(token):
		return token

	for prefix in PREFIX_SET:
		if len(token) - len(prefix) >= 3 and \
			count_vowel(token[len(prefix):]) >= 2:

			if prefix == ('i') and check_consonant(token[2]):
				continue

			if '-' in token:	
				token = token.split('-')

				if token[0] == prefix and check_vowel(token[1][0]):
					PREFIX.append(prefix)
					return token[1]

				token = '-'.join(token)

			if token[0: len(prefix)] == prefix:
				if count_vowel(token[len(prefix):]) >= 2:
					# if check_vowel(token[len(token) - len(prefix) - 1]):
				# 	continue

					if prefix == 'panganga':
						PREFIX.append(prefix)
						return 'ka' + token[len(prefix):]
					
					PREFIX.append(prefix)
					return token[len(prefix):]

	return token


def clean_duplication(token, DUPLICATE):
	"""
		Checks token for duplication. (ex. araw-araw = araw)
			token: word to be stemmed duplication
		returns STRING
	"""

	if check_validation(token):
		return token

	if '-' in token and token.index('-') != 0 and \
		token.index('-') != len(token) -  1:

		split = token.split('-')

		if all(len(tok) >= 3 for tok in split):
			if split[0] == token[1] or split[0][-1] == 'u' and change_letter(split[0], -1, 'o') == split[1] or \
				split[0][-2] == 'u' and change_letter(split[0], -2, 'o')  == split[1]:
				DUPLICATE.append(split[0])
				return split[0]

			elif split[0] == split[1][0:len(split[0])]:
				DUPLICATE.append(split[1])
				return split[1]

			elif split[0][-2:] == 'ng':
				if split[0][-3] == 'u':
					if split[0][0:-3] + 'o' == split[1]:
						DUPLICATE.append(split[1])
						return split[1]

				if split[0][0:-2] == split[1]:
					DUPLICATE.append(split[1])
					return split[1]

		else:
			return '-'.join(split)
	
	return token


def clean_repitition(token, REPITITION):
	"""
		Checks token for repitition. (ex. nakakabaliw = nabaliw)
			token: word to be stemmed repitition
		returns STRING
	"""

	if check_validation(token):
		return token

	if len(token) >= 4:
		if check_vowel(token[0]):
			if token[0] == token[1]:
				REPITITION.append(token[0])
				return token[1:]

		elif check_consonant(token[0]) and count_vowel(token) >= 2:
			if token[0: 2] == token[2: 4] and len(token) - 2 >= 4:
				REPITITION.append(token[2:4])
				return token[2:]
			
			elif token[0: 3] == token[3: 6] and len(token) - 3 >= 4:
				REPITITION.append(token[3:6])
				return token[3:]

	return token


def clean_stemmed(token, CLEANERS, REPITITION):
		
	if not token:
		return ""
	
	"""
		Checks for left-over affixes and letters.
			token: word to be cleaned for excess affixes/letters
		returns STRING
	"""

	global PERIOD_FLAG
	global PASS_FLAG

	CC_EXP = ['dr', 'gl', 'gr', 'ng', 'kr', 'kl', 'kw', 'ts', 'tr', 'pr', 'pl', 'pw', 'sw', 'sy'] # Consonant + Consonant Exceptions

	if token[-1] == '.' and PASS_FLAG == False:
		PERIOD_FLAG = True

	if not check_vowel(token[-1]) and not check_consonant(token[-1]):
		CLEANERS.append(token[-1])
		token = token[0:-1]

#	if not check_vowel(token[0]) and not check_consonant(token[0]):
#		CLEANERS.append(token[0])
#		token = token[1:]

	if check_validation(token):
		return token

	if len(token) >= 3 and count_vowel(token) >= 2:
		token = clean_repitition(token,	REPITITION)

		if check_consonant(token[-1]) and token[- 2] == 'u':
			CLEANERS.append('u')
			token = change_letter(token, -2, 'o')

		if token[len(token) - 1] == 'u':
			CLEANERS.append('u')
			token = change_letter(token, -1, 'o')

		if token[-1] == 'r':
			CLEANERS.append('r')
			token = change_letter(token, -1, 'd')

		if token[-1] == 'h' and check_vowel(token[-1]):
			CLEANERS.append('h')
			token = token[0:-1]

		# if token[0] == 'i':
		# 	token = token[1:]

		if token[0] == token[1]:
			CLEANERS.append(token[0])
			token = token[1:]

		if (token[0: 2] == 'ka' or token[0: 2] == 'pa') and check_consonant(token[2]) \
			and count_vowel(token) >= 3:
			
			CLEANERS.append(token[0: 2])
			token = token[2:]

		if(token[-3:]) == 'han' and count_vowel(token[0:-3]) == 1:
			CLEANERS.append('han')
			token = token[0:-3] + 'i'

		if(token[-3:]) == 'han' and count_vowel(token[0:-3]) > 1:
			CLEANERS.append('han')
			token = token[0:-3]

		if len(token) >= 2 and count_vowel(token) >= 3:
			if token[-1] == 'h' and check_vowel(token[-2]):
				CLEANERS.append('h')
				token = token[0:-1]

		if len(token) >= 6 and token[0:2] == token[2:4]:
			CLEANERS.append('0:2')
			token = token[2:]

		if any(REP[0] == 'r' for REP in REPITITION):
			CLEANERS.append('r')
			token = change_letter(token, 0, 'd')

		if token[-2:] == 'ng' and token[-3] == 'u':
			CLEANERS.append('u')
			token = change_letter(token, -3, 'o')

		if token[-1] == 'h':
			CLEANERS.append('h')
			token = token[0:-1]

		if any(token[0:2] != CC for CC in CC_EXP) and check_consonant(token[0:2]):
			CLEANERS.append(token[0:2])
			token = token[1:]

	return token


def tg_stemmer(tokens):

    global PERIOD_FLAG
    global PASS_FLAG

    pre_stem     = inf_stem = suf_stem = rep_stem = \
        du1_stem = du2_stem = cle_stem = '-'
    word_info    = {}
    PREFIX     = []
    INFIX      = []
    SUFFIX     = []
    DUPLICATE  = []
    REPITITION = []
    CLEANERS   = []

    word_info['clean'] = '-'
    stemmed_tokens = []

    for token in tokens:
        word_info = {}
        word_info["word"] = token

        if (PERIOD_FLAG == True and token[0].isupper()) or \
                (PERIOD_FLAG == False and token[0].islower()):
            token = token.lower()
            du1_stem = clean_duplication(token, DUPLICATE)
            pre_stem = clean_prefix(du1_stem, PREFIX)
            rep_stem = clean_repitition(pre_stem, REPITITION)
            inf_stem = clean_infix(rep_stem, INFIX)
            rep_stem = clean_repitition(inf_stem, REPITITION)
            suf_stem = clean_suffix(rep_stem, SUFFIX)
            du2_stem = clean_duplication(suf_stem, DUPLICATE)
            cle_stem = clean_stemmed(du2_stem, CLEANERS, REPITITION)
            cle_stem = clean_duplication(cle_stem, DUPLICATE)

            if '-' in cle_stem:
                cle_stem.replace('-', '')

        else:
            PERIOD_FLAG = False
            cle_stem = clean_stemmed(token, CLEANERS, REPITITION)
            word_info["root"]   = token
            word_info["prefix"] = '[]'
            word_info["infix"]  = '[]'
            word_info["suffix"] = '[]'
            word_info["repeat"] = '[]'
            word_info["dupli"]  = '[]'
            word_info["clean"]  = cle_stem

        stemmed_tokens.append(cle_stem)

    return stemmed_tokens



In [55]:
#ENGLISH STEMMER

from nltk.stem import WordNetLemmatizer

def english_lemmatizer(token):
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(token)



In [67]:
from langdetect import detect
import pandas as pd



lemmatized_tokens = []
for row in train['tokenize']:
    lemmatized_row = []
    for token in row:
        try: 
            language = detect(token)
        except:
            continue
        if language == 'tl':
            lemmatized_token = tg_stemmer(token)
        elif language == 'en':
            lemmatized_token = english_lemmatizer(token)
        else:
            lemmatized_token = token
        lemmatized_row.appendstr(lemmatized_token)
    lemmatized_tokens.append(' '.join(lemmatized_row))

train['lemmatize_modstem'] = pd.Series(lemmatized_tokens)


AttributeError: 'list' object has no attribute 'appendstr'

In [58]:
train[['label','tokenize','lemmatize_modstem']].head(5)

,label,tokenize,lemmatize_modstem
0,0,"[inaasahan, vice, president, jejomar, binay, t...",inaasahan vice president jejomar y g
1,1,"[mar, roxas, tang, ina, tuwid, daan, daw, eh, ...",mar roxas g ina tuwid daan daw eh a a di straight
2,0,"[salamat, sawang, suporta, taga, makati, pagba...",t g suporta a makati k y in makati
3,1,"[putangina, mo, binay, takbo]",a mo y takbo
4,0,"[binay, with, selective, amnesia, forgetting, ...",y with selective amnesia forgetting about the ...


In [68]:
#tokenization 

import nltk
from nltk.tokenize import WhitespaceTokenizer
train['lemmatize_modstem'] = train['lemmatize_modstem'].apply(nltk.tokenize.WhitespaceTokenizer().tokenize) 
train['lemmatize_modstem'].head(10)

TypeError: expected string or bytes-like object, got 'list'

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train['lemmatize_modstem'], train['label'], test_size=0.3, random_state=1)

In [ ]:
# Vectorize the sentences using TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(map(' '.join, X_train))
X_test = vectorizer.transform(map(' '.join, X_test))

In [ ]:
# Train the SVM model
clf = svm.SVC()
clf.fit(X_train, y_train)

# Test the model on the testing set
y_pred = clf.predict(X_test)

# Evaluate the model's performance
accuracy = clf.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.7249122807017544
